In [1]:
# check out https://huggingface.co/microsoft/Phi-3.5-vision-instruct for more details

import torch
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig
from PIL import Image
import requests

model_id = "yifeihu/TB-OCR-preview-0.1"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [2]:
# Note: set _attn_implementation='eager' if you don't have flash_attn installed
model = AutoModelForCausalLM.from_pretrained(
  model_id, 
  device_map="cuda", 
  trust_remote_code=True, 
  torch_dtype="auto", 
  _attn_implementation='eager',
  quantization_config=BitsAndBytesConfig(load_in_4bit=False) # Optional: Load model in 4-bit mode to save memory
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
processor = AutoProcessor.from_pretrained(model_id, 
  trust_remote_code=True, 
  num_crops=16
)

c:\users\yushl\desktop\ws\yifeihu\env\lib\site-packages\transformers\models\auto\image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [10]:
def phi_ocr(image1):
    question = "Convert the text to markdown format."
    question = "extract text from image into a json output." # this is required
    
    prompt_message = [{
        'role': 'user',
        'content': f'<|image_1|>\n{question}',
    }]

    prompt = processor.tokenizer.apply_chat_template(prompt_message, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt, [image1], return_tensors="pt").to("cuda") 

    generation_args = { 
        "max_new_tokens": 1024, 
        "temperature": 0.1, 
        "do_sample": False
    }

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args
    )

    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

    response = response.split("<image_end>")[0] # remove the image_end token 

    return response

In [11]:
import fitz

def load_pdf(im):
    doc = fitz.open(im)
    page = doc[0]
    zoom = 1
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat)
    return Image.frombytes('RGB', [pix.width, pix.height], pix.samples)

In [12]:
%%time
test_image_url = "https://huggingface.co/yifeihu/TB-OCR-preview-0.1/resolve/main/sample_input_1.png?download=true"
#image = Image.open(requests.get(test_image_url, stream=True).raw)

image = Image.open('invoice7.png').convert('RGB')
image = load_pdf('2023-form-5500.pdf')
#image = load_pdf('t5008-23b.pdf')

response = phi_ocr(image)
print(response)


**Form 5500**

*Annual Return/Report of Employee Benefit Plan*

2023

*Department of the Treasury*

*Internal Revenue Service*

*Department of Labor*

*Employer Benefit Service*

*Administration*

*Pension Benefit Guaranty Corporation*

*Part I* Annual Report Identification Information

*For calendar plan year 2023 or fiscal plan year beginning*

*A this return/report is for:*

*A single-employer plan (Firms checking this box must provide participating employer information)*

*B This return/report is:*

*A single-employer plan*

*A DFE (specify—*

*A single-employer plan*

*A the final return/report (less than 12 months)*

*C If the plan is collectively-bargained check here…*

*D Check box if filing under:*

*A Form 558*

*A automatic extension*

*E If this is a retroactively adopted plan permitted by SECURE Act section 201, check here…*

*D Check box if filing under:*

*A Form 558*

*A special extension (enter description)*

*E If this is a retroactively adopted plan permitted by SECU